<img src="https://raw.githubusercontent.com/seismo-live/seismo_live/refs/heads/master/notebooks/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/images/obspy_logo_full_524x179px.png" width=90%> 

In [1]:
try:
  import obspy
  import cartopy
except ModuleNotFoundError:
  !pip -qq install obspy cartopy
  import obspy
  import cartopy

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

ObsPy has clients to directly fetch data via...

- FDSN webservices (IRIS, Geofon/GFZ, USGS, NCEDC, SeisComp3 instances, ...)
- ArcLink (EIDA, ...)
- Earthworm
- SeedLink (near-realtime servers)
- NERIES/NERA/seismicportal.eu
- NEIC
- SeisHub (local seismological database)

This introduction shows how to use the FDSN webservice client. The FDSN webservice definition is by now the default web service implemented by many data centers world wide. Clients for other protocols work similar to the FDSN client.


#### Station Metadata

The FDSN client can be used to request station metadata. Stations can be looked up using a wide range of constraints (see [ObsPy documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_stations.html)):

 * network/station code
 * time range of operation
 * geographical (lonlat-box, circular by distance)

Let's start by finding some interesting stations. We will look at the tsunamigenic eruption on 22 December 2018. You can find the date from the [Global Volcanism Program](https://volcano.si.edu/volcano.cfm?vn=262000). We request station metadata from within 6 degrees of the event.



In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

t = UTCDateTime(2018, 12, 22, 13, 55, 0)
lon = 105.435 
lat = -6.149

client = Client("Geofon")
inventory = client.get_stations(
    starttime=t-100, endtime=t+100,
    longitude=lon, latitude=lat, maxradius=6,
    matchtimeseries=None, level="station")
print(inventory)

The **`level=...`** keyword is used to specify the level of detail in the requested inventory

- `"network"`: only return information on networks matching the criteria
- `"station"`: return information on all matching stations
- `"channel"`: return information on available channels in all stations networks matching the criteria
- `"response"`: include instrument response for all matching channels (large result data size!)

#### Waveform Data

Only the stations from the GE network are openly available. In the following we will download the 1 Hz data from the following stations:

* GE.BBJI..LHZ
* GE.MNAI..BHZ
* GE.PMBI..BHZ
* GE.SMRI..BHZ
* GE.UGM..BHZ


In [ ]:
import os
from obspy import UTCDateTime, Stream, read
from obspy.clients.fdsn import Client

fout = 'anak_krakatau_2018.mseed'
if not os.path.isfile(fout):
    client = Client("Geofon")
    st = Stream()
    for station in ["GE.BBJI", "GE.MNAI", "GE.PMBI", "GE.SMRI", "GE.UGM"]:
      net, stat = station.split(".")
      st_ = client.get_waveforms(net, stat, "*", "LHZ",
                                t, t + 10 * 60)
      st_.write(f"{station}..LHZ.mseed", format='MSEED')
      st += st_
    st.write(fout, format='MSEED')
else:
  st = read(fout)

print(st)
st.plot();

- again, waveform data is returned as a Stream object
- for all custom processing workflows it does not matter if the data originates from a local file or from a web service



For waveform requests that include instrument correction, the appropriate instrument response information can be attached to waveforms automatically:     
(Of course, for work on large datasets, the better choice is to download all station information and avoid the internal repeated webservice requests)

In [ ]:
t = UTCDateTime("2018-12-22T13:56:48")  # Tohoku
st = client.get_waveforms("GE", "BBJI", "*", "LHZ",
                          t, t + 10 * 60, attach_response=True)
st.plot();

st.remove_response()
st.plot();

#### Event Metadata

The FDSN client can also be used to request event metadata:

In [ ]:
t = UTCDateTime("2018-12-22T13:50:00")
client = Client("Geofon")
catalog = client.get_events(starttime=t, endtime=t + 3600,
                            minmagnitude=2)
print(catalog)
catalog.plot();

Requests can have a wide range of constraints (see [ObsPy Documentation](http://docs.obspy.org/packages/autogen/obspy.fdsn.client.Client.get_events.html)):

- time range
- geographical (lonlat-box, circular by distance)
- depth range
- magnitude range, type
- contributing agency

All data requested using the FDSN client can be directly saved to file using the **`filename="..."`** option. The data is then stored exactly as it is served by the data center, i.e. without first parsing by ObsPy and outputting by ObsPy.

In [ ]:
client.get_events(starttime=t-100, endtime=t+3600, minmagnitude=5,
                  filename="/tmp/requested_events.xml")
client.get_stations(network="GE", station="BBJI", level="response",
                    filename="/tmp/requested_stations.xml")
client.get_waveforms("GE", "BBJI", "*", "LHZ", t, t + 10 * 60,
                     filename="/tmp/requested_waveforms.mseed")
!ls -lrt /tmp/requested*

#### FDSN Client Exercise

Use the FDSN client to assemble a waveform dataset for on event.

- search for a large earthquake (e.g. by depth or in a region of your choice, use option **`limit=5`** to keep network traffic down)

- search for stations to look at waveforms for the event. stations should..
    * be available at the time of the event
    * have a vertical 1 Hz stream ("LHZ", to not overpower our network..)
    * be in a narrow angular distance around the event (e.g. 90-91 degrees)
    * adjust your search so that only a small number of stations (e.g. 3-6) match your search criteria

- for each of these stations download data of the event, e.g. a couple of minutes before to half an hour after the event
- put all data together in one stream (put the `get_waveforms()` call in a try/except/pass block to silently skip stations that actually have no data available)
- print stream info, plot the raw data

- correct the instrument response for all stations and plot the corrected data

If you have time, assemble and plot another similar dataset (e.g. like before stations at a certain distance from a big event, or use Transportable Array data for a big event, etc.)